:In this lab, we are going to build a seq2seq model for translating text from English to Catalan without attention

In [ ]:
from __future__ import unicode_literals, print_function, division

import re
import math
import random
import string
import unicodedata
from io import open

import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns

To ensure reproducibility of the experiments, we can set the seed to a fixed number.

In [ ]:
seed = 123
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def plot_attention(attention, xtitle="Keys", ytitle="Queries"):
    """ Plots the attention map."""

    sns.set(rc={'figure.figsize':(12, 8)})
    ax = sns.heatmap(
        attention.detach().cpu(),
        linewidth=0.5,
        cmap="Blues",
        square=True)

    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.set_xlabel(xtitle)
    ax.set_ylabel(ytitle)

    plt.show()

In [ ]:
!wget http://www.manythings.org/anki/cat-eng.zip
!unzip cat-eng.zip && rm cat-eng.zip
!mkdir data
!mv "cat.txt" "data/eng-cat.txt"

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

## Data pre-processing

First we need to pre-process the raw text data. We need to make sure to lowercase all of it, remove all non-letter characters, and limit the maximum lenght of the sentences to MAX_LENGTH. Eventually, we create ```pairs```, a list where each component has a sentence and its translation, both pre-processed.



In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
import unicodedata

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Remove extra info
    new_lines = []
    for line in lines:
        new_lines.append(line[0:(line.find('CC-BY')-1)])


    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in new_lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 20

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'cat', False)
print(random.choice(pairs))

We have a total of 1306 pairs of sentences containing 1436 english words and 1773 catalan words. We use a very small dataset to reduce training time, but a larger dataset would be necessary to learn something meaningful.

## Building the seq2seq model

We proceed to build our seq2seq model with attention. First we define our encoder RNN:

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # TODO: Define the Embedding matrix (use nn.Embedding)
        self.embedding = nn.Embedding(input_size, hidden_size)

        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Now we define our decoder, with an attention mechanism (```self.attn```) built inside. Try to map the code with the following diagram:

<p align="center"><br>
<img src="https://github.com/telecombcn-dl/labs-all/raw/main/labs/attention/images/attention_tensor_dance_last.jpeg?raw=true" class="center" title="Decoding with attention" width="800"/>
</p><br>

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True)
        self.dropout = nn.Dropout(self.dropout_p)
        # Removed attention mechanism
        # self.attn = AdditiveAttention(self.hidden_size, self.hidden_size, self.hidden_size // 2)
        # self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)

        # Only use the hidden state from the GRU for the output
        output, hidden = self.gru(embedded, hidden)

        # Removed attention computation and combination
        # context, attn_weights = self.attn(query=x, key=encoder_outputs, value=encoder_outputs)
        # x_w_context = torch.cat((x, context), dim=-1)
        # x_w_context = self.attn_combine(x_w_context)

        output = F.log_softmax(self.out(output), dim=-1)
        # Return None for attention weights as they are no longer computed
        return output, hidden, None

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

    # First hidden state used by the decoder is the last encoder hidden state
    decoder_hidden = encoder_hidden

    # We feed the decoder with the whole target sentence (teacher forcing),
    # first we append SOS_token
    decoder_input = torch.cat([
        torch.tensor([[SOS_token]], device=device),
        target_tensor[:, :-1]
    ], dim=1)


    # Modified to not receive attention weights
    decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, encoder_outputs)

    loss = criterion(
        decoder_output.view(-1, decoder_output.size(-1)),
        target_tensor.view(-1)
    )
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_tensor.numel()

In [ ]:
import time

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # Random sample of n_iters pairs
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 50000, print_every=3000)

In [ ]:
# def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
#     with torch.no_grad():
#         input_tensor = tensorFromSentence(input_lang, sentence)
#         encoder_hidden = encoder.initHidden()

#         encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

#         decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

#         decoder_hidden = encoder_hidden



#         decoded_words = []
#         decoder_attentions = []

#         for di in range(max_length):
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
#             decoder_attentions.append(decoder_attention.squeeze(0).data)
#             topv, topi = decoder_output.data.topk(1)
#             if topi.squeeze().item() == EOS_token:
#                 decoded_words.append('<EOS>')
#                 break
#             else:
#                 decoded_words.append(output_lang.index2word[topi.squeeze().item()])
#             decoder_input = topi.detach().squeeze(1)

#         decoder_output, decoder_hidden = decoder(
#             decoder_input, decoder_hidden, encoder_outputs
#         )
#         return decoded_words, None
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_hidden = encoder.initHidden()

        encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # (1,1)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for _ in range(max_length):
            # Cho phép decoder trả (out, hid) hoặc (out, hid, attn)
            ret = decoder(decoder_input, decoder_hidden, encoder_outputs)
            if isinstance(ret, (tuple, list)):
                if len(ret) == 3:
                    decoder_output, decoder_hidden, _ = ret  # bỏ qua attention
                elif len(ret) == 2:
                    decoder_output, decoder_hidden = ret
                else:
                    raise ValueError(f"Decoder returned {len(ret)} values, expected 2 or 3.")
            else:
                raise ValueError("Decoder must return (output, hidden) or (output, hidden, attn).")

            # decoder_output shape: (1, 1, vocab_size)
            topv, topi = decoder_output.data.topk(1)     # topi: (1,1,1)

            # Lấy id để kiểm tra EOS
            idx = topi.squeeze().item()                  # scalar
            if idx == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx])

            # Làm input cho bước sau, giữ shape (1,1)
            decoder_input = topi.squeeze(-1).detach()    # (1,1)

        return decoded_words, None


In [ ]:
def sentence_accuracy(pred_tokens, ref_tokens):
    # cắt theo độ dài nhỏ hơn
    min_len = min(len(pred_tokens), len(ref_tokens))
    correct = sum(1 for i in range(min_len) if pred_tokens[i] == ref_tokens[i])
    return correct / max(len(ref_tokens), 1)


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluateRandomly(encoder, decoder, n=10):
    smoothie = SmoothingFunction().method4
    for i in range(n):
        pair = random.choice(pairs)
        print('Source:', pair[0])
        print('Reference:', pair[1])

        output_words, _ = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words[:-1])

        print('Model:', output_sentence)

        # tokenized reference and hypothesis
        ref_tokens = pair[1].split()
        hyp_tokens = output_sentence.split()

        # Accuracy
        acc = sentence_accuracy(hyp_tokens, ref_tokens)
        # BLEU (nghiêng về n-gram, nên dùng smoothing cho câu ngắn)
        bleu = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)

        print(f'Accuracy: {acc:.2f}, BLEU: {bleu:.2f}\n')


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)